In [14]:
import json
import time
import datetime
import re
import os
import sys
import subprocess
import pandas as pd
import pytesseract
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image
import base64
import cv2 as cv
from itertools import zip_longest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
import threading

"""
请求网页
"""

no = 1

chrome_options = Options()
# chrome_options.add_argument('--headless') #无界面
# chrome_options.add_argument('blink-settings=imagesEnabled=false') #不加载图片
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) #防检测
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()  #最大化窗口
# driver.set_window_size(1920,1080)  #设置分辨率
driver.get("https://www.mytoken.io/zh/latest/")

#跳过公告弹窗
try:
    page_value = '/html/body/div[2]/div/div[2]/div/div[2]/button/span'
    driver.implicitly_wait(30)  #隐式等待5秒
    driver.find_element(by=By.XPATH,value=page_value).click()
except:
    pass

#按24h增长率排序
time.sleep(2)
page_value = "/html/body/div/div/main/div/div/div[3]/table/thead/tr/th[5]/div/div/img[2]"
driver.find_element(by=By.XPATH,value=page_value).click()

#一般信息
time.sleep(2)
page_value = f"/html/body/div/div/main/div/div/div[3]/table/tbody/tr[{no}]"
driver.implicitly_wait(30)  #隐式等待5秒
coin = driver.find_element(by=By.XPATH,value=page_value).text.split('\n')[:-3]
ans = ['序号','名称1','名称2','价格','24h','7d']
res = dict(zip_longest(ans,coin))

#跳转到详细页面
page_value = f"/html/body/div/div/main/div/div/div[3]/table/tbody/tr[{no}]"
driver.implicitly_wait(30)  #隐式等待5秒
driver.find_element(by=By.XPATH,value=page_value).click()
time.sleep(5)

#跳过公告弹窗
try:
    page_value = '/html/body/div[2]/div/div[2]/div/div[2]/button/span'
    driver.find_element(by=By.XPATH,value=page_value).click()
except:
    pass


page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[2]/div/div"
for i in driver.find_elements(by=By.XPATH,value=page_value):
    basedata = i.text.split('\n')
    res[basedata[0]] = basedata[1]
    
page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[3]/div/div"
for i in driver.find_elements(by=By.XPATH,value=page_value):
    links = i.text.split('\n')
    res[links[0]] = links[1]
    
# # 所属链
# time.sleep(2)
# try:
#     page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[3]/div/div[1]"
#     chain = WebDriverWait(driver,30,0.5).until(lambda x:x.find_element(by=By.XPATH,value=page_value)).text
#     print(chain)
#     if chain.split('\n')[0] == '基础链':
#         res['所属链'] = chain.split('\n')[-1]
# except:
#     pass

# #合约
# try:
#     page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[3]/div/div[2]"
#     driver.implicitly_wait(30)  #隐式等待5秒
#     contract = driver.find_element(by=By.XPATH,value=page_value).text
#     if contract.split('\n')[0] in ['BSC','Ethereum','Polygon']:
#         res['合约'] = contract.split('\n')[1]
# except:
#     pass

# #链接
# links = []
# try:
#     page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[4]/div/div"
#     driver.implicitly_wait(30)  #隐式等待5秒
#     link_list = driver.find_elements(by=By.XPATH,value=page_value)
#     length = len(link_list)
#     for i in range(length):
#         if link_list[i].text.split('\n')[0] == '官网':
#             page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[4]/div/div[{i+1}]/a"
#             res['官网'] = driver.find_element(by=By.XPATH,value=page_value).get_attribute('href')

#         elif link_list[i].text.split('\n')[0] == '社交媒体':
#             page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[4]/div/div[{i+1}]/div/a"
#             for j in (driver.find_elements(by=By.XPATH,value=page_value)):
#                 link = j.get_attribute('href')
#                 if 'twitter' in link:
#                     res['推特'] = link
#                 elif 't.me' in link:
#                     res['电报'] = link
#                 else:
#                     res['其它链接'] = link
#             #     links.append(j.get_attribute('href'))
#             # res['相关链接'] = links
# except:
    # pass

In [15]:
res

{'序号': '1',
 '名称1': 'KFA',
 '名称2': 'Crypto KFA',
 '价格': '$0.32348038',
 '24h': '102.16%',
 '7d': '151.77%',
 '换手率': '0.05%',
 '上架交易所数量': '1',
 '流通量': '2,000,000,000 KFA',
 '当前总供应量': '2,000,000,000 KFA',
 '流通率': '100%',
 '最大供应量': '2,000,000,000 KFA',
 '全球总市值占比': '<0.01%',
 '24小时成交额': '249736.03238351998',
 '官网': 'https://www.kfacoin.com/',
 '白皮书': 'https://www.kfacoin.com/',
 '区块浏览器': 'https://tronscan.org/#/token20/THScBUvbuvkiJnrva6kc45DSwy2AS22tPm'}

In [13]:
page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[3]/div/div"
for i in driver.find_elements(by=By.XPATH,value=page_value):
    print(i.text.split('\n'))

['官网', 'https://www.kfacoin.com/']
['白皮书', 'https://www.kfacoin.com/']
['区块浏览器', 'https://tronscan.org/#/token20/THScBUvbuvkiJnrva6kc45DSwy2AS22tPm']
